In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import scipy.stats as sps
from sympy import *
from matplotlib import cm
plt.style.use('ggplot')
%matplotlib inline

def scan_from_csv(filename,header=None):
    if not header:
        return pd.read_csv(filename,na_values='None')
    else:
        return pd.read_csv(filename,na_values='None',header=-1)

def write_answer_to_file(answer,file):
    with open(file, 'w') as answer_file:  
        answer_file.write(answer)

In [36]:
data = np.array(scan_from_csv('sample2.csv',header=1)[0])

In [6]:
alpha = 0.05

In [37]:
n = len(data)
print(len(data))

122


In [39]:
from statsmodels.sandbox.stats.multicomp import multipletests  # Holm method
from skidmarks import wald_wolfowitz # Wald-Wollfowitz test

In [49]:
def get_stats(statf,n,k=100):
    stats = np.zeros(k)
    gens = sps.uniform.rvs(size=(k,n))
    for i in range(k):
        stats[i] = statf(gens[i])
    stats = np.sort(stats)
    return np.array(stats)

In [41]:
def run_inversions_stat(x):
    #  simple O(N^2) realisation
    inv = 0
    n = len(x)
    for i in range(n):
        for j in range(i+1,n):
            inv += (x[i] > x[j])
    return inv

def run_ww_stat(x):
    return wald_wolfowitz(x > np.median(x))['n_runs']  # return stat

In [42]:
%%time
inv_stats = get_stats(run_inversions_stat,n,1000)
ww_stats = get_stats(run_ww_stat, n, 1000)

CPU times: user 14 s, sys: 23.3 ms, total: 14 s
Wall time: 14 s


In [56]:
v_inv_high = inv_stats[int((len(inv_stats) - 1)*(1 - alpha/2))]
v_inv_low = inv_stats[int((len(inv_stats) - 1)*alpha/2)]
v_ww_high = ww_stats[int((len(inv_stats) - 1)*(1 - alpha/2))]
v_ww_low = ww_stats[int((len(inv_stats) - 1)*alpha/2)]

Проверим сложную гипотезу, что данные это выборка с помощью метода Бонферрони для $\alpha = 0.05 * 2 = 0.1$.
При этом в качестве статистик будем использовать $\{Inv > u_{1-\frac{\alpha}{2}} \cup  Inv < u_{\frac{\alpha}{2}}\}, \{WW_{runs} > v_{1-\frac{\alpha}{2}} \cup WW_{runs} < v_{\frac{\alpha}{2}}\}$, где 
пороги уже найдены моделированием для $\alpha = 0.05$

In [57]:
bounds = [v_inv, v_ww]
st_inv = run_inversions_stat(data)
st_ww = run_ww_stat(data)
print(st_inv > v_inv_high or st_inv < v_inv_low, st_ww > v_ww_high or st_ww < v_ww_low)

True True


Как видно сложная гипотеза отвергнута. Видна явная закономерность:

In [47]:
print(data > np.median(data))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True False  True  True  True  True  True False  True
  True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False]
